In [10]:
from pathlib import Path
import wandb
import torch

model_id = "model-k1knlvtk:v0"
model_path = Path("artifacts") / model_id

api = wandb.Api()
artifact = api.artifact("ain-space/gaia/" + model_id, type="model")
if model_path.exists():
    print("Model already downloaded")
else:
    artifact_dir = artifact.download(model_path)

ckpt = artifact_dir + "/model.ckpt"
checkpoint = torch.load(ckpt, weights_only=True, map_location="cpu")

Model already downloaded


In [ ]:
model = yaml2model("gaia_ae_8_trunc_normal.yaml")

input = torch.randn(1, 1, 343)
model(input)

In [ ]:
import torch

ckpt = artifact_dir + "/model.ckpt"
checkpoint = torch.load(ckpt, weights_only=True, map_location="cpu")
checkpoint

In [ ]:
model.load_state_dict(checkpoint["state_dict"])
model.eval()

input = torch.randn(10, 1, 343)
model(input)

In [ ]:
import os

os.makedirs("gaia-calibrated-v0", exist_ok=True)

export_options = torch.onnx.ExportOptions(dynamic_shapes=False)
onnx = torch.onnx.dynamo_export(
    model.variational_encoder,
    torch.randn(256, 1, 343, device="cpu"),
    export_options=export_options,
)
onnx.save("gaia-calibrated-v0/encoder.onnx")

onnx = torch.onnx.dynamo_export(
    model.decoder, torch.randn(256, 3, device="cpu"), export_options=export_options
)
onnx.save("gaia-calibrated-v0/decoder.onnx")

In [ ]:
import onnxruntime as ort

ort_session = ort.InferenceSession("gaia-calibrated-v0/encoder.onnx")

In [ ]:
#!rsync gaia-calibrated-v0 space:/var/www/html/space/models/